In [ ]:
import pandas as pd
pd.set_option('display.max_rows', 200)

In [ ]:
apple_data_path = "../data/applemobilitytrends-2020-06-06.csv"
google_data_path = "../data/Global_Mobility_Report.csv"

In [ ]:
apple_data = pd.read_csv(apple_data_path)
google_data = pd.read_csv(google_data_path)

In [ ]:
apple_data.head()

In [ ]:
google_data.head()

## Group Apple Data by County

In [ ]:
apple_data["geo_type"].unique()

In [ ]:
# Set up grouping by country 
by_county = apple_data[apple_data["geo_type"] == "county"].copy()
by_county.head()

In [ ]:
# Make sure US is the only country 
by_county["country"].unique()

In [ ]:
# We only have driving data for this? 
by_county["transportation_type"].unique()

In [ ]:
# drop unecessary columns 
by_county.drop(["geo_type", "alternative_name"], axis=1, inplace=True)

In [ ]:
# Melt date columns to rows 
apple_data_melted = by_county.melt(id_vars=["region", "transportation_type", 
                                            "sub-region", "country"], 
                                  var_name="date",
                                  value_name="mobility_from_baseline")

In [ ]:
apple_data_melted.head()

In [ ]:
# rename cols 
apple_data_melted.rename({"sub-region": "state",
                          "region": "county"}, 
                         axis=1, inplace=True)
# apple_data_melted.set_index(["county", "date"], inplace=True)
# apple_data_melted.sort_index().head()
# apple_data_melted.reset_index(inplace=True)
apple_data_melted.head()

## Group Google Data by County

In [ ]:
google_data["sub_region_1"].unique()

In [ ]:
google_data["sub_region_2"].unique()

In [ ]:
# sub_region 2 is all US counties
google_by_county = google_data[~google_data["sub_region_2"].isna()].copy()
google_by_county.head() # We want this format for the Apple Data

In [ ]:
# rename cols, drop unecessary cols 
google_by_county.rename({"sub_region_2":"county",
                         "sub_region_1":"state",
                         "country_region":"country"},
                        axis=1, inplace=True)
google_by_county.drop("country_region_code", axis=1, inplace=True)


google_new_index = google_by_county.set_index(["county", "date"]) # no longer needed 
google_by_county.head()

## Grouping Data

In [ ]:
google_by_county.reset_index(inplace=True, drop=True)
google_by_county.shape

In [ ]:
apple_data_melted.reset_index(inplace=True, drop=True)
apple_data_melted.shape

In [ ]:
cols_to_include = google_new_index.columns.difference(apple_data_melted.columns) # just want metrics from google
cols_to_include

In [ ]:
# inner vs outer has some weird effects ?
# merged_data = apple_data_melted.join(google_new_index[cols_to_include], how='inner')
# #merged_data.drop(["country_region", "country_region_code"], axis=1, inplace=True)
# merged_data.head()

In [ ]:
apple_data_melted.head()

In [ ]:
# inner join drops apple data up to 2/15, so baseline needs to be re-normalized
merged_data = pd.merge(google_by_county, apple_data_melted, on=["county", "state", "date", "country"]) 

### Notes on baselines:

Apple and Google calculated baselines differently. Google's baseline is on a per-weekday basis, while Apple uses a specific day in January as the baseline. Will they need to be re-calculated to be in the same units or does it not matter ? 

In [ ]:
merged_data.dropna().head(50)

In [ ]:
merged_data[merged_data["state"] == "Florida"]

In [ ]:
merged_data.shape